Import Packages and Data

In [ ]:
import pandas as pd
from MicroStrategyConnection import *
from mstrio.project_objects import SuperCube

CSV_Folder_Path = 'C:/Micha/MSTR/MicroStrategyPython/DataExport/'
#df_Import_Table = pd.read_csv(CSV_Folder_Path + 'test_retail_data_including_ids.csv', sep=';', nrows=1000)
df_Import_Table = pd.read_csv(CSV_Folder_Path + 'test_retail_data_including_ids.csv', sep=';')

Prep data for our scenario: drop old ID columns

In [ ]:
columns_to_drop = ['Call Center ID', 'Category ID', 'Customer ID', 'Item ID', 'Supplier ID', 'Category ID', 'Month ID', 'Region ID', 'Subcategory ID']
df_Master_Table = df_Import_Table.drop(columns = columns_to_drop)

Create Lookup Table and create ID Columns in Lookup Table

In [ ]:
df_LU_Customer = df_Master_Table[['Customer', 'Customer Age']].drop_duplicates()
df_LU_Customer.reset_index(drop=True, inplace=True)
df_LU_Customer.insert(0, 'Customer ID', range(1, 1 + len(df_LU_Customer)))

#We want only the "old" column to join with lookup table in our master table
df_Master_Table.drop(columns=['Customer Age'], inplace=True)

Create new Main Table by joining via old column with new Lookup Table. Leave new ID Column and drop all DESC columns that are available in Lookup Table

In [ ]:
df_Master_Table = df_Master_Table.join(df_LU_Customer.set_index('Customer'), on='Customer')
df_Master_Table.drop(columns=['Customer', 'Customer Age'], inplace=True)

Write iCube and configure Attribute Columns

In [ ]:
mstr_conn = GetMicroStrategyConnection()
iCube = SuperCube(connection=mstr_conn, name='_iCube_Test_STAR_Schema_Transformation')
iCube.add_table(name='master_table', data_frame=df_Master_Table, update_policy='replace', to_attribute=['Customer ID', 'Month Number'])
iCube.add_table(name='lu_customer', data_frame=df_LU_Customer, update_policy='replace', to_attribute=['Customer ID', 'Customer Age'])
iCube.create()
mstr_conn.close()